# DuckDBの利用

ライブラリ インストール
- pip install duckdb

In [1]:
from pprint import pprint
import json
import re
import numpy as np
import pandas as pd
pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 20)

import duckdb

## Create DataFrame

データスキーマ: 料理名, 調理時間(分), 食材のカラム...])　量は2人前    
ingredients_df = pd.DataFrame(columns=["料理名","調理時間(分)","塩","こしょう", ...])  

In [2]:
# サンプルデータを読み込む
# 通常はここでいろいろデータを加工する
ingredients_df = pd.read_csv('ingredients.csv').fillna('')
display(ingredients_df.head())

,料理名,調理時間(分),豚ロース肉,塩・こしょう,小麦粉,パン粉,サラダ油（揚げ用）,生姜すりおろし,醤油,みりん,...,イカ,練りごま,ラー油,鶏がらスープ,しょうが,あじ(小),レモングラス,ナンプラー,ココナッツミルク,こぶみかんの葉
0,アイスバイン,150,,,,,,,,,...,,,,,,,,,,
1,アサリのクリームクラムチャウダー,30,,,大さじ1,,,,,,...,,,,,,,,,,
2,あじの南蛮漬け,40,,,,,,,大さじ2,,...,,,,,,4尾,,,,
3,エビグラタン,35,,少々,大さじ2,,,,,,...,,,,,,,,,,
4,エビチリ,20,,,,,,,,,...,,,,,,,,,,


## Create DuckDB

DBの作成、持ち方、保存のバリエーション

#### csvファイルを読んで、DuckDB をインメモリで利用

```python
# DuckDB のインメモリ接続を作成
con = duckdb.connect(database=':memory:')

# CSV ファイルをロードしてテーブルに登録
con.execute("""
    CREATE TABLE ingredients AS
    SELECT * FROM read_csv_auto('ingredients_df.csv')
""")
```

#### pandas DataFrameを元に、ストレージ上に永続化したDuckDBを作成して利用

```python
# DuckDB ファイルを作成（永続化DBとして）
con = duckdb.connect("ingredients.duckdb")

# DataFrameをDuckDB内にテーブルとして保存
con.register("ingredients_df", ingredients_df)
con.execute("""
    CREATE TABLE ingredients AS
    SELECT * FROM ingredients_df;
""")
```

####  pandas DataFrameを元にインメモリでDBを作成, DuckDBをストレージへも作成

```python
# 1. pandas DataFrameを元にインメモリでDuckDBを作成 (以降、インメモリで処理を継続)
# 2. DuckDB を後でconnectできる状態でストレージへ保存 (永続化)

# 1. DuckDB をインメモリに作成 --------------------------------
con = duckdb.connect(database=':memory:')

# DuckDBを作成
con.execute("""
    CREATE TABLE ingredients AS
    SELECT * FROM ingredients_df
""")

# 2. DuckDB を後でconnectできる状態でストレージへ保存 -----------
# ファイルDBをバックアップ用にアタッチ
con.execute("ATTACH 'ingredients.duckdb' AS disk_db;")

# ファイル側にテーブルをコピー（DuckDB内部形式で保存）
con.execute("""
    CREATE TABLE disk_db.ingredients AS
    SELECT * FROM ingredients;
""")
# ただし、インデックスは保存されないので、列ごとにインデックス作成が必要
# これらのインデックスはストレージ上で永続化され、再利用が可能
# con.execute("CREATE INDEX idx_name ON disk_db.ingredients(col_name);")
```

In [3]:
# DuckDB ====================
# pandas DataFrameを元に、インメモリでDuckDBを作成

# DuckDB のインメモリ接続を作成
con = duckdb.connect(database=':memory:')

# DataFrameをインメモリテーブルに登録
con.execute("""
    CREATE TABLE ingredients AS
    SELECT * FROM ingredients_df
""")

# SQLクエリのサンプル実行
# 卵を使った料理
sql_result = con.execute("""
    SELECT 料理名, 卵
    FROM ingredients
    WHERE 卵 <> ''
""").fetchdf()  # pandas DataFrame で受け取る
print(f"sql_result ---")
display(sql_result)

sql_result ---


,料理名,卵
0,エビフライ,1個
1,カニチャーハン,2個
2,たこ焼き,1個
3,ふかひれスープ,1個
4,広島風お好み焼き,2個
5,大阪風豚玉お好み焼き,2個
6,茶碗蒸し,2個
7,明石風たこ焼き,1個


## Select DB

In [4]:
# --- 複数の列名で検索 ---

def choice_ingredients(target_mats):
    select_clause = ', '.join(['料理名'] + target_mats)
    where_clause  = ' AND '.join([f"{c} <> ''" for c in target_mats])
    # --- SELECT文を生成 ---
    sql = f"""
    SELECT {select_clause}
    FROM ingredients
    WHERE {where_clause}
    """
    #print(sql)

    # --- SQL実行 ---
    sql_result = con.execute(sql).fetchdf().fillna('')
    print(f"sql_result ---")
    display(sql_result)

choice_ingredients(['卵'])
choice_ingredients(['卵','三つ葉'])
choice_ingredients(['玉ねぎ','にんじん'])

sql_result ---


,料理名,卵
0,エビフライ,1個
1,カニチャーハン,2個
2,たこ焼き,1個
3,ふかひれスープ,1個
4,広島風お好み焼き,2個
5,大阪風豚玉お好み焼き,2個
6,茶碗蒸し,2個
7,明石風たこ焼き,1個


sql_result ---


,料理名,卵,三つ葉
0,茶碗蒸し,2個,適量
1,明石風たこ焼き,1個,少々


sql_result ---


,料理名,玉ねぎ,にんじん
0,アイスバイン,1個,1本
1,あじの南蛮漬け,1/2個,1/3本
2,ミネストローネ,1/2個,1/2本
3,肉じゃが,1個,1/2本


In [5]:
# 動的SQL生成 =============
# fixed_wordを含んだカラム名を対象に検索する
fixed_word = '卵'

all_cols = [c[1] for c in con.execute("PRAGMA table_info('ingredients')").fetchall()]
target_cols = sorted([c for c in all_cols if fixed_word in c]) # fixed_wordを含むカラム名

# --- SELECT句とWHERE句を動的に作成 ---
cols_select = ', '.join(['料理名'] + target_cols)
where_clause = ' OR '.join([f"{c} <> ''" for c in target_cols])
# --- SELECT文を生成 ---
sql = f"""
SELECT {cols_select}
FROM ingredients
WHERE {where_clause}
"""
print(sql)
# --- SQL実行 ---
sql_result = con.execute(sql).fetchdf().fillna('')
print(f"sql_result ---")
display(sql_result)


SELECT 料理名, 卵, 卵黄, 温泉卵
FROM ingredients
WHERE 卵 <> '' OR 卵黄 <> '' OR 温泉卵 <> ''

sql_result ---


,料理名,卵,卵黄,温泉卵
0,エビフライ,1個,,
1,カニチャーハン,2個,,
2,シーザーサラダ,,,1個
3,たこ焼き,1個,,
4,タリアテッレ・カルボナーラ,,2個,
5,ふかひれスープ,1個,,
6,広島風お好み焼き,2個,,
7,大阪風豚玉お好み焼き,2個,,
8,茶碗蒸し,2個,,
9,明石風たこ焼き,1個,,


In [6]:
sql = f"""
SELECT 料理名, 玉ねぎ, ピーマン, 鶏むね肉, 鶏もも肉
FROM ingredients
WHERE 玉ねぎ <> '' AND ピーマン <> '' AND (鶏むね肉 <>'' OR 鶏もも肉 <>'') 
"""
#print(sql)

# --- SQL実行 ---
sql_result = con.execute(sql).fetchdf().fillna('') # <= No Match
print(f"sql_result ---")
display(sql_result)

sql_result ---


,料理名,玉ねぎ,ピーマン,鶏むね肉,鶏もも肉
